In [1]:
# Ignore insignificant warnings (ex: deprecation warnings)
import warnings
warnings.filterwarnings('ignore')

In [6]:
from rdflib import Graph, Namespace, URIRef, Literal, BNode
from rdflib.namespace import RDF, RDFS, OWL, XSD, SKOS, NamespaceManager
import os

g = Graph()
g.parse('ttl/ThesaurusInferred.owl')

<Graph identifier=Nab86df552c5d42879ab778d6ba37dd29 (<class 'rdflib.graph.Graph'>)>

In [16]:
entities = []
relations = []
for s,p,o in g.triples((None,RDF.type,OWL.Class)):
    for _,_,l in g.triples((s,RDFS.label,None)):
        entities.append(l)     
domain = ""
range = ""
for s,p,o in g.triples((None,RDF.type,OWL.ObjectProperty)):
    for _,_,l in g.triples((s,RDFS.label,None)):
        relation = l
    for _,_,d in g.triples((s,RDFS.domain,None)):
        for _,_,l in g.triples((d,RDFS.label,None)):
            domain = l
    for _,_,r in g.triples((s,RDFS.range, None)):
        for _,_,l in g.triples((r,RDFS.label,None)):
            range = l
    relations.append(f"{relation}({domain},{range})")

for s,p,o in g.triples((None,RDF.type,OWL.DatatypeProperty)):
    for _,_,l in g.triples((s,RDFS.label,l)):
        relation = l
    for _,_,d in g.triples((s,RDFS.domain,None)):
        for _,_,l in g.triples((d,RDFS.label,None)):
            domain = l
    for _,_,r in g.triples((s,RDFS.range, None)):
        for _,_,l in g.triples((r,RDFS.label,None)):
            range = l
    relations.append(f"{relation}({domain},{range})")


In [17]:
len(entities)

184165

In [19]:
relations

['Disease_Has_Associated_Anatomic_Site(Disease, Disorder or Finding,Anatomic Structure, System, or Substance)',
 'Disease_Has_Primary_Anatomic_Site(Disease, Disorder or Finding,Anatomic Structure, System, or Substance)',
 'Disease_Has_Metastatic_Anatomic_Site(Disease, Disorder or Finding,Anatomic Structure, System, or Substance)',
 'Disease_Has_Normal_Tissue_Origin(Disease, Disorder or Finding,Anatomic Structure, System, or Substance)',
 'Disease_Has_Normal_Cell_Origin(Disease, Disorder or Finding,Anatomic Structure, System, or Substance)',
 'Disease_Has_Abnormal_Cell(Disease, Disorder or Finding,Abnormal Cell)',
 'Disease_Has_Molecular_Abnormality(Disease, Disorder or Finding,Molecular Abnormality)',
 'Disease_Has_Cytogenetic_Abnormality(Disease, Disorder or Finding,Molecular Abnormality)',
 'Disease_Has_Finding(Disease, Disorder or Finding,Disease, Disorder or Finding)',
 'Disease_Is_Grade(Disease, Disorder or Finding,Property or Attribute)',
 'Disease_May_Have_Normal_Tissue_Origin(D

In [20]:
text_sentence = """
Can we reduce different cancers to a set of common traits? 
"""

In [26]:
prompt = f"""
Given the following ontology relations and sentences, please extract the triples from the sentence according to the relations in the ontology. 
In the output, use the information from the extracted triples, and generate an answer to the question: {text_sentence} 
in the given Example Output format.

CONTEXT:

Ontology Relations: {relations}

Example Sentence: How can knowing the mutations that cause a patient's cancer shape treatment?

Example Output: Scientists have spent decades learning the precise genetic mutations 
that can cause a cell to start dividing uncontrollably. Cancer patients can now have 
their tumors sequenced to identify the genetic root of their disease—and we have 
targeted treatments for many of these genetic alterations—but we are still very much 
in the early years of making the most of genomic data.
"""

In [27]:
from openai import OpenAI
client = OpenAI()
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": prompt},
    {"role": "user", "content": text_sentence},
  ]
)

In [28]:
print(response.choices[0].message.content)

Scientists are investigating the various traits that different cancers share. Research has shown that certain molecular abnormalities and genetic mutations can be common across various cancer types. By identifying these shared characteristics, it may be possible to develop standardized treatment approaches that target these common traits. Thus, understanding the underlying similarities can lead researchers closer to a unified model for cancer treatment.


In [ ]:
In 2000, researchers Robert Weinberg and Douglas Hanahan proposed the "Hallmarks of Cancer" in a landmark Cell review. Their short list of underlying principles 
that define a cancer includes evasion of cell death, the ability to grow self-sufficiently, insensitivity to antigrowth signals, tissue invasion and metastasis, 
limitless cell division potential, and the development of blood vessels.
Since the paper (which is the most-downloaded and most-cited paper ever published in Cell) was published, new studies have added two emerging hallmarks—abnormal 
cell metabolism and evasion of the immune system. 
"The notion that cancer was a single disease (as might be concluded from the notion of a common set of hallmarks) was of course an illusion," says Weinberg, who 
studies the molecular mechanisms of cancer at the Whitehead Institute of Biomedical Research and MIT. "While the general principles of cancer formation may extend 
widely over many tumor types, the detailed behaviors of cancer cause each tumor to be a unique entity—a unique invention of nature." 